In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 6: Análise de regressão múltipla: problemas adicionais (Multiple Regression Analysis: Further Issues)
Exemplo 6.3: Efeitos da frequência escolar no desempenho de exames finais
             (EFFECTS OF ATTENDANCE ON FINAL EXAM PERFORMANCE)
             
Arquivo com os dados: attend.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import sympy as sp

In [2]:
df = pd.read_excel('attend.xls',
                   header=None,
                   usecols=[2, 3, 5, 10],
                   names=['priGPA', 'ACT', 'atndrte', 'stndfnl' ])

In [3]:
df.head()

,priGPA,ACT,atndrte,stndfnl
0,2.64,23,84.375,0.472689
1,3.52,25,68.750,0.052521
2,2.46,24,93.750,0.892857
3,2.61,20,96.875,0.262605
4,3.32,23,100.000,1.733193


### Comandos sugeridos

#### Regressão por Mínimos Quadrados Ordinários (MQO ou OLS na sigla em inglês)

In [4]:
# Regressão pelo Método de Mínimos Quadrados Ordinários (MQO ou OLS na sigla em inglês)
# Inclui uma interação entre priGPA e atndrte (priGPA:atndrte)
modelo = smf.ols(formula='stndfnl ~ atndrte + priGPA + ACT + np.square(priGPA) + np.square(ACT) + priGPA:atndrte', data=df)
reg = modelo.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                stndfnl   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     33.25
Date:                Fri, 22 Jul 2022   Prob (F-statistic):           3.49e-35
Time:                        19:28:52   Log-Likelihood:                -868.90
No. Observations:                 680   AIC:                             1752.
Df Residuals:                     673   BIC:                             1783.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.0503      1.360      1.507      0.132      -0.621       4.721
atndrte              -0.0067      0.010     -0.656      0.512      -0.027       0.013
priGPA               -1.6285      0.481     -3.386      0.001      -2.573      -0.684
ACT                  -0.1280      0.098     -1.300      0.194      -0.321       0.065
np.square(priGPA)     0.2959      0.101      2.928      0.004       0.097       0.494
np.square(ACT)        0.0045      0.002      2.083      0.038       0.000       0.009
priGPA:atndrte        0.0056      0.004      1.294      0.196      -0.003       0.014
==============================================================================
Omnibus:                        2.581   Durbin-Watson:                   2.279
Prob(Omnibus):                  0.275   Jarque-Bera (JB):                2.474
Skew:                          -0.095   Prob(JB):                        0.290
Kurtosis:                       3.226   Cond. No.                     2.43e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.43e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Testes de hipóteses individuais e conjuntas referentes B1 e B6

In [5]:
reg.pvalues

Intercept            0.132225
atndrte              0.512005
priGPA               0.000751
ACT                  0.194047
np.square(priGPA)    0.003523
np.square(ACT)       0.037634
priGPA:atndrte       0.196173
dtype: float64

In [ ]:
'''
Teste de hipóteses individual (Teste t)
H0: B_atndrte = 0                B_atndrte: B1
H1: B_atndrte != 0

Valor p = 51,20%. 
Valor p > 10%. Assim, atndrte é individualmente não significante se considerarmos qualquer nível padrão de nível de 
significância (1%, 5% ou 10%).
'''

In [ ]:
'''
Teste de hipóteses individual (Teste t)
H0: B_priGPA:atndrte = 0                (B_priGPA:atndrte): B6
H1: B_priGPA:atndrte != 0

Valor p = 19,62%. 
Valor p > 10%. Assim, priGPA:atndrte (termo de interação) é individualmente não significante se considerarmos qualquer nível 
padrão de nível de significância (1%, 5% ou 10%).
'''

In [ ]:
'''
Teste de hipóteses conjuntas (Teste F)
H0: B_atndrte = 0, B_priGPA:atndrte = 0             B_atndrte: B1; (B_priGPA:atndrte): B6
H1: pelo menos um dos parâmetros na hipótese nula é diferente de zero
'''

In [6]:
hipotese_nula = 'atndrte = 0, priGPA:atndrte = 0'
teste_F = reg.f_test(hipotese_nula)
teste_F.pvalue

array(0.01368385)

In [ ]:
'''
Conclusão
Como o valor p = 1,37%, rejeitamos H0 no nível de 5%.
Assim, "atndrte" e "priGPA:atndrte" são conjuntamente significantes no nível de significância de 5% (ainda que, individualmente,
não são significantes como vimos pelos testes t acima).

'''

#### Efeito parcial de atndrte sobre stndfnl (no valor médio de priGPA)

In [7]:
# Média amostral de priGPA
media_priGPA = df['priGPA'].mean()
media_priGPA

2.5867750000000016

In [8]:
# Estabelece variáveis como se fossem "variáveis matemáticas"
B = sp.symbols('B:7', real=True)    # B será uma tupla com as variáveis simbólicas B0, B1, ..., B6
stndfnl, atndrte, priGPA, ACT = sp.symbols('stndfnl, atndrte, priGPA, ACT', real=True)    

In [9]:
# Estabelece a função stndfnl (não vamos incluir o termo de erro)
stndfnl = B[0] + B[1] * atndrte + B[2] * priGPA + B[3] * ACT + B[4] * priGPA**2 + B[5]*ACT**2 + B[6] * priGPA*atndrte
stndfnl

ACT**2*B5 + ACT*B3 + B0 + B1*atndrte + B2*priGPA + B4*priGPA**2 + B6*atndrte*priGPA

In [10]:
# Apenas cria um dicionário para facilitar encontrar as correspondências entre os parâmetros e as respectivas variáveis
parametros = dict()
for (k, var) in enumerate(reg.params.index):
    parametros[B[k]] = var
parametros

{B0: 'Intercept',
 B1: 'atndrte',
 B2: 'priGPA',
 B3: 'ACT',
 B4: 'np.square(priGPA)',
 B5: 'np.square(ACT)',
 B6: 'priGPA:atndrte'}

In [11]:
# derivada de stndfnl em relação à atndrte
dstndfnl_datndrte = stndfnl.diff(atndrte)
dstndfnl_datndrte

B1 + B6*priGPA

In [12]:
# Efeito parcial de atndrte sobre stndfnl no valor médio de priGPA
dstndfnl_datndrte.subs({B[1]: reg.params['atndrte'], B[6]: reg.params['priGPA:atndrte'], priGPA:media_priGPA})

0.00773655711196323

In [ ]:
'''
No valor médio de priGPA (= 2.59), o efeito de atndrte sobre stndfnl é 0.0077.
'''

#### Significância estatística do efeito parcial de atndrte sobre stndfnl calculado acima

In [13]:
# derivada de stndfnl em relação à atndrte
dstndfnl_datndrte

B1 + B6*priGPA

In [ ]:
'''
Queremos saber se a estimativa que obtivemos acima do efeito parcial de atndrte sobre stndfnl (=0.0077) na média de priGPA  é 
estatisticamente diferente de zero. Assim, fazemos o seguinte teste de hipóteses

H0: B1 + B6 * priGPA = 0
H1: B1 + B6 * priGPA != 0
(para priGPA no seu valor amostral médio de 2.59)

'''

In [14]:
# Lembrando que B1 é o parâmetro associado à variável abaixo
parametros[B[1]]

'atndrte'

In [15]:
# Lembrando que B6 é o parâmetro associado à variável abaixo
parametros[B[6]]

'priGPA:atndrte'

In [16]:
# Para escrever a hipótese, substituímos B1 e B6 por suas respectivas variáveis: 'atndrte' e 'priGPA:atndrte'
hipotese_nula = 'atndrte + priGPA:atndrte * 2.59 = 0'   # 2.59 é a média amostral de priGPA
teste_t = reg.t_test(hipotese_nula)

In [17]:
# Erro padrão
teste_t.sd

array([[0.00263929]])

In [18]:
teste_t.tvalue

array([[2.93812549]])

In [19]:
teste_t.pvalue

array(0.00341499)

In [ ]:
'''
Como o valor p (=0,3%) é menor do que 1%, concluímos que, na priGPA média, o efeito de atndrte sobre stndfnl é positivo e 
estatisticamente significante (em qualquer nível de significância padrão).
'''

#### Efeito parcial de priGPA sobre stndfnl

In [20]:
dstndfnl_dpriGPA = stndfnl.diff(priGPA)
dstndfnl_dpriGPA

B2 + 2*B4*priGPA + B6*atndrte

#### Observação: veja o que ocorre se utilizamos asterisco (*) no termo de interação, e não dois pontos (:) como no exemplo acima

In [21]:
modelo_ = smf.ols(formula='stndfnl ~ np.square(ACT) + priGPA*atndrte', data=df)
reg_ = modelo_.fit()
reg_.params

Intercept        -0.241642
np.square(ACT)    0.001858
priGPA           -0.569470
atndrte          -0.020711
priGPA:atndrte    0.011484
dtype: float64

In [ ]:
'''
Embora só temos "np.square(ACT)" e "priGPA*atndrte" do lado direito da fórmula, a regressão é executada com "priGPA" e 
"atndrte" inseridas individualmente, além do termo de interação priGPA*atndrte.
'''